# 👋 Getting started 2: Training adversarially robust 1-Lipschitz neural networks for classification


The goal of this series of tutorials is to show the different usages of `deel-lip`.

In the first notebook, we have shown how to create 1-Lipschitz neural networks with
`deel-lip`. In this second notebook, we will show how to train adversarially robust
1-Lipschitz neural networks with `deel-lip`.

In particular, we will cover the following:

1. [📚 Theoretical background](#theoretical_background) A brief theoretical background
   on adversarial robustness. This section can be safely skipped if one is not
   interested in the theory.

2. [💪 Training provable adversarially robust 1-Lipschitz neural networks on the MNIST dataset](#deel_keras)
   Using the MNIST dataset, we will show examples of training adversarially robust
   1-Lipschitz neural networks using `deel-lip` loss functions
   `TauCategoricalCrossentropy` and `MulticlassHKR`.

We will also see that:

- when training robust models, there is an accuracy-robustness trade-off
- the `MulticlassKR` loss function can be used to assess the adversarial robustness of
  the resulting models

## 📚 Theoretical background <a id='theoretical_background'></a> <a name='theoretical_background'></a>

### Adversarial attacks

In the context of classification problems, an adversarial attack is the result of adding
an _adversarial perturbation_ $\epsilon$ to the input data point $x$ of a trained
predictive model $A$, with the intent to change its prediction (for simplicity, $A$
returns a class as opposed to a set of logits in the formalism used below).

In simple mathematical terms, an adversarial example (i.e. a successful adversarial
attack) can be transcribed as below:

$$A(x)=y_1,$$

$$A(x+\epsilon)=y_{\epsilon},$$

where:

$$y_1\neq y_\epsilon.$$


### An adversarial example

The following example is directly taken from
https://adversarial-ml-tutorial.org/introduction/.

![pigs.png](../assets/pigs.png)

The first image is correctly classified as a **pig** by a classifier. The second image
is incorrectly classified as an **airplane** by the same classifier.

While both images cannot be distinguished from our (human) perspective, the second image
is in fact the result of surimposing "noise" (i.e. adding an adversarial perturbation)
to the original first image.


Below is a visualization of the added noise, zoomed-in by a factor of 50 so that we can
see it:

![noise.png](../assets/noise.PNG)


### Adversarial robustness of 1-Lipschitz neural network

The adversarial robustness of a predictive model is its ability to remain accurate and
reliable when subjected to adversarial perturbations.

A major advantage of 1-Lipschitz neural networks is that they can offer provable
guarantees on their robustness for any particular input $x$, by providing a
_certificate_ $\epsilon_x$. Such a guarantee can be understood by using the following
terminology:

> "For an input $x$, we can certify that there are no adversarial perturbations
> constrained to be under the certificate $\epsilon_x$ that will change our model's
> prediction."

In simple mathematical terms:

For a given $x$, $\forall \epsilon$ such that $\|\epsilon\|<\epsilon_x$, we obtain that:

$$A(x)=y,$$

$$A(x+\epsilon)=y_{\epsilon},$$

then:

$$y_{\epsilon}=y.$$

💡 We will use certificates in this notebook as a metric to evaluate the provable
adversarial robustness of deep learning 1-Lispchitz models.

💡 Depending on the type of norm you choose (e.g. $L_1$ or $L_2$), the guarantee you can
offer will differ, as $\|\epsilon\|_2<\epsilon_x$ and $\|\epsilon\|_1<\epsilon_x$ are
not equivalent.

🚨 **Note**: _`deel-lip` only deals with $L_2$ norm, as previously said in the first
notebook 'Getting started 1'_

As such, an additional example of guarantee that could be obtained with `deel-lip` with
a more precise formulation would be:

> "For an input $x$, we can certify that are no adversarial perturbations constrained to
> be within a $L_2$-norm ball of certificate $\epsilon_{x,L_2}$ that will change our
> model's prediction."

For a given $x$, $\forall \epsilon$ such that $\|\epsilon\|_2<\epsilon_{x,L_2}$, we
obtain that: $$A(x)=y,$$ $$A(x+\epsilon)=y_{\epsilon},$$ then: $$y_{\epsilon}=y.$$

## 💪 Training provable adversarially robust 1-Lipschitz neural networks on the MNIST dataset <a id='deel_keras'></a> <a name='deel_keras'></a>

### 💾 MNIST dataset

MNIST dataset contains a large number of 28x28 handwritten digit images to which are
associated digit labels.


In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
# Load MNIST Database
(X_train, y_train_ord), (X_test, y_test_ord) = mnist.load_data()

# standardize and reshape the data
X_train = np.expand_dims(X_train, -1) / 255
X_test = np.expand_dims(X_test, -1) / 255

# one hot encode the labels
y_train = to_categorical(y_train_ord)
y_test = to_categorical(y_test_ord)

### 🎮 Control over the accuracy-robustness trade-off with `deel-lip`'s loss functions.

When training neural networks, there is always a compromise between the robustness and
the accuracy of the models. In simple terms, achieving stronger robustness often
involves sacrificing some performance (at the extreme point, the most robust function
being the constant function).

In this section, we will show the pivotal role of `deel-lip`'s loss functions in
training 1-Lipschitz networks. Each of these functions comes with its own set of
hyperparameters, enabling you to precisely navigate and adjust the balance between
accuracy and robustness.

We show two cases. In the first case, we use `deel-lip`'s `TauCategoricalCrossentropy`
from the `losses` submodule. In the second case, we use another loss function from
`deel-lip`: `MulticlassHKR`.

#### 🔮 Prediction Model

Since we will be instantiating the same model four times within our examples, we
encapsulate the code for creating the model within a function to enhance conciseness:


In [3]:
from deel import lip

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Flatten


def create_conv_model(name_model, input_shape, output_shape):
    """
    A simple convolutional neural network, made to be 1-Lipschitz.
    """
    model = lip.Sequential(
        [
            Input(shape=input_shape),
            lip.layers.SpectralConv2D(
                filters=16,
                kernel_size=(3, 3),
                use_bias=True,
                kernel_initializer="orthogonal",
            ),
            lip.layers.GroupSort2(),
            lip.layers.ScaledL2NormPooling2D(
                pool_size=(2, 2), data_format="channels_last"
            ),
            lip.layers.SpectralConv2D(
                filters=32,
                kernel_size=(3, 3),
                use_bias=True,
                kernel_initializer="orthogonal",
            ),
            lip.layers.GroupSort2(),
            lip.layers.ScaledL2NormPooling2D(
                pool_size=(2, 2), data_format="channels_last"
            ),
            Flatten(),
            lip.layers.SpectralDense(
                64,
                use_bias=True,
                kernel_initializer="orthogonal",
            ),
            lip.layers.GroupSort2(),
            lip.layers.SpectralDense(
                output_shape,
                activation=None,
                use_bias=False,
                kernel_initializer="orthogonal",
            ),
        ],
        name=name_model,
    )

    return model

In [4]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[-1]

#### Cross-entropy loss: `TauCategoricalCrossentropy`


Similar to the classes we have seen in "Getting started 1", the
`TauCategoricalCrossentropy` class inherits from its equivalent in `keras`, but it comes
with an additional settable parameter named 'temperature' and denoted as: `tau`. This
parameter will allow to adjust the robustness of our model. The lower the temperature
is, the more robust our model becomes, but it also becomes less accurate.

To show the impact of the parameter `tau` on both the performance and robustness of our
model, we will train two models on the MNIST dataset. The first model will have a
temperature of 100, the second model will have a temperature of 3.

<u>Note</u>: The performance achieved in this tutorial is not state-of-the-art. It is
presented solely for educational purposes. Performance can be enhanced by employing a
different network architecture or by training for additional epochs.


In [ ]:
# high-temperature model
model_1 = create_conv_model("cross_entropy_model_1", input_shape, output_shape)

temperature_1 = 100.0

model_1.compile(
    loss=lip.losses.TauCategoricalCrossentropy(tau=temperature_1),
    optimizer=Adam(1e-4),
    # notice the use of lip.losses.Certificate_Multiclass,
    # to assess adversarial robustness
    metrics=[
        "accuracy",
        lip.metrics.CategoricalProvableAvgRobustness(disjoint_neurons=False),
    ],
)

In [6]:
# low-temperature model
model_2 = create_conv_model("cross_entropy_model_2", input_shape, output_shape)

temperature_2 = 3.0

model_2.compile(
    loss=lip.losses.TauCategoricalCrossentropy(tau=temperature_2),
    optimizer=Adam(1e-4),
    metrics=[
        "accuracy",
        lip.metrics.CategoricalProvableAvgRobustness(disjoint_neurons=False),
    ],
)

💡 Notice that we use the accuracy metric to measure the performance, and we use the
`Certificate_Multiclass` loss to measure adversarial robustness. The latter is a measure
of our model's average certificates: **the higher this measure is, the more robust our
model is**.

**🚨 Note:** _This is true only for 1-Lipschitz neural networks_


We fit both our models and observe the results.


In [7]:
# fit the high-temperature model
result_1 = model_1.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=10,
    validation_data=(X_test, y_test),
    shuffle=True,
    # verbose=1,
)

Epoch 1/10
235/235 [==============================] - 6s 10ms/step - loss: 0.0104 - accuracy: 0.7916 - CategoricalProvableAvgRobustness: 0.0290 - val_loss: 0.0028 - val_accuracy: 0.9184 - val_CategoricalProvableAvgRobustness: 0.0392
Epoch 2/10
235/235 [==============================] - 2s 8ms/step - loss: 0.0024 - accuracy: 0.9291 - CategoricalProvableAvgRobustness: 0.0414 - val_loss: 0.0019 - val_accuracy: 0.9443 - val_CategoricalProvableAvgRobustness: 0.0426
Epoch 3/10
235/235 [==============================] - 2s 8ms/step - loss: 0.0017 - accuracy: 0.9491 - CategoricalProvableAvgRobustness: 0.0451 - val_loss: 0.0014 - val_accuracy: 0.9574 - val_CategoricalProvableAvgRobustness: 0.0471
Epoch 4/10
235/235 [==============================] - 2s 7ms/step - loss: 0.0013 - accuracy: 0.9592 - CategoricalProvableAvgRobustness: 0.0481 - val_loss: 0.0011 - val_accuracy: 0.9658 - val_CategoricalProvableAvgRobustness: 0.0512
Epoch 5/10
235/235 [==============================] - 2s 7ms/step - los

In [8]:
# fit the low-temperature model
result_2 = model_2.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=10,
    validation_data=(X_test, y_test),
    shuffle=True,
    # verbose=1,
)

Epoch 1/10
235/235 [==============================] - 4s 10ms/step - loss: 0.3420 - accuracy: 0.7768 - CategoricalProvableAvgRobustness: 0.2767 - val_loss: 0.1553 - val_accuracy: 0.9009 - val_CategoricalProvableAvgRobustness: 0.4921
Epoch 2/10
235/235 [==============================] - 2s 8ms/step - loss: 0.1320 - accuracy: 0.9126 - CategoricalProvableAvgRobustness: 0.5823 - val_loss: 0.1054 - val_accuracy: 0.9314 - val_CategoricalProvableAvgRobustness: 0.6599
Epoch 3/10
235/235 [==============================] - 2s 8ms/step - loss: 0.1022 - accuracy: 0.9321 - CategoricalProvableAvgRobustness: 0.6845 - val_loss: 0.0880 - val_accuracy: 0.9437 - val_CategoricalProvableAvgRobustness: 0.7208
Epoch 4/10
235/235 [==============================] - 2s 8ms/step - loss: 0.0892 - accuracy: 0.9419 - CategoricalProvableAvgRobustness: 0.7354 - val_loss: 0.0794 - val_accuracy: 0.9481 - val_CategoricalProvableAvgRobustness: 0.7705
Epoch 5/10
235/235 [==============================] - 2s 8ms/step - los

In [9]:
# metrics for the high-temperature model => performance-oriented
print(f"Model accuracy: {result_1.history['val_accuracy'][-1]:.4f}")
print(
    f"Model's mean certificate: {result_1.history['val_CategoricalProvableAvgRobustness'][-1]:.4f}"
)
print(f"Loss' temperature: {model_1.loss.tau.numpy():.1f}")

Model accuracy: 0.9806
Model's mean certificate: 0.0623
Loss' temperature: 100.0


In [10]:
# metrics for the low-temperature model => robustness-oriented
print(f"Model accuracy: {result_2.history['val_accuracy'][-1]:.4f}")
print(
    f"Model's mean certificate: {result_2.history['val_CategoricalProvableAvgRobustness'][-1]:.4f}"
)
print(f"Loss' temperature: {model_2.loss.tau.numpy():.1f}")

Model accuracy: 0.9621
Model's mean certificate: 0.8598
Loss' temperature: 3.0


When decreasing the temperature, we observe a large increase in robustness, but a slight
decrease in accuracy.


#### Hinge-Kantorovich–Rubinstein loss: `MulticlassHKR`


We work in the same way as in the previous section. The difference lies in the
parameters that control the robustness.

We count two of them: `min_margin` (minimal margin) and `alpha` (regularization factor).

As will be shown in the following, a higher minimal margin and a lower alpha increases
robustness.


In [11]:
# performance-oriented model
model_3 = create_conv_model("HKR_model_3", input_shape, output_shape)

min_margin_3 = 0.01
alpha_3 = 1000

model_3.compile(
    loss=lip.losses.MulticlassHKR(min_margin=min_margin_3, alpha=alpha_3),
    optimizer=Adam(1e-4),
    metrics=[
        "accuracy",
        lip.metrics.CategoricalProvableAvgRobustness(disjoint_neurons=False),
    ],
)

In [12]:
# robustness-oriented model
model_4 = create_conv_model("HKR_model_4", input_shape, output_shape)

min_margin_4 = 0.2
alpha_4 = 50

model_4.compile(
    loss=lip.losses.MulticlassHKR(min_margin=min_margin_4, alpha=alpha_4),
    optimizer=Adam(1e-4),
    metrics=[
        "accuracy",
        lip.metrics.CategoricalProvableAvgRobustness(disjoint_neurons=False),
    ],
)

We fit both our models and observe the results.


In [13]:
# fit the model
result_3 = model_3.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=10,
    validation_data=(X_test, y_test),
    shuffle=True,
    # verbose=1,
)

Epoch 1/10
235/235 [==============================] - 4s 10ms/step - loss: 6.4450 - accuracy: 0.7472 - CategoricalProvableAvgRobustness: 0.0258 - val_loss: 1.7008 - val_accuracy: 0.9056 - val_CategoricalProvableAvgRobustness: 0.0361
Epoch 2/10
235/235 [==============================] - 2s 7ms/step - loss: 1.4576 - accuracy: 0.9168 - CategoricalProvableAvgRobustness: 0.0372 - val_loss: 1.0670 - val_accuracy: 0.9411 - val_CategoricalProvableAvgRobustness: 0.0411
Epoch 3/10
235/235 [==============================] - 2s 6ms/step - loss: 0.9756 - accuracy: 0.9410 - CategoricalProvableAvgRobustness: 0.0410 - val_loss: 0.7664 - val_accuracy: 0.9550 - val_CategoricalProvableAvgRobustness: 0.0431
Epoch 4/10
235/235 [==============================] - 2s 7ms/step - loss: 0.7388 - accuracy: 0.9532 - CategoricalProvableAvgRobustness: 0.0441 - val_loss: 0.6139 - val_accuracy: 0.9621 - val_CategoricalProvableAvgRobustness: 0.0462
Epoch 5/10
235/235 [==============================] - 2s 8ms/step - los

In [14]:
# fit the model
result_4 = model_4.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=10,
    validation_data=(X_test, y_test),
    shuffle=True,
    # verbose=1,
)

Epoch 1/10
235/235 [==============================] - 4s 9ms/step - loss: 1.9754 - accuracy: 0.8073 - CategoricalProvableAvgRobustness: 0.1097 - val_loss: 0.5700 - val_accuracy: 0.9194 - val_CategoricalProvableAvgRobustness: 0.1783
Epoch 2/10
235/235 [==============================] - 2s 8ms/step - loss: 0.3412 - accuracy: 0.9265 - CategoricalProvableAvgRobustness: 0.2227 - val_loss: 0.0606 - val_accuracy: 0.9375 - val_CategoricalProvableAvgRobustness: 0.2713
Epoch 3/10
235/235 [==============================] - 2s 9ms/step - loss: -0.0592 - accuracy: 0.9404 - CategoricalProvableAvgRobustness: 0.3110 - val_loss: -0.2699 - val_accuracy: 0.9468 - val_CategoricalProvableAvgRobustness: 0.3581
Epoch 4/10
235/235 [==============================] - 2s 9ms/step - loss: -0.3596 - accuracy: 0.9476 - CategoricalProvableAvgRobustness: 0.3993 - val_loss: -0.5494 - val_accuracy: 0.9537 - val_CategoricalProvableAvgRobustness: 0.4502
Epoch 5/10
235/235 [==============================] - 2s 9ms/step - 

In [15]:
# performance-oriented model
print(f"Model accuracy: {result_3.history['val_accuracy'][-1]:.4f}")
print(
    f"Model's mean certificate: {result_3.history['val_CategoricalProvableAvgRobustness'][-1]:.4f}"
)
print(f"Loss' minimum margin: {model_3.loss.min_margin.numpy():.2f}")
print(f"Loss' alpha: {model_3.loss.alpha.numpy():.1f}")

Model accuracy: 0.9739
Model's mean certificate: 0.0590
Loss' minimum margin: 0.01
Loss' alpha: 1000.0


In [16]:
# robustness-oriented model
print(f"Model accuracy: {result_4.history['val_accuracy'][-1]:.4f}")
print(
    f"Model's mean certificate: {result_4.history['val_CategoricalProvableAvgRobustness'][-1]:.4f}"
)
print(f"Loss' minimum margin: {model_4.loss.min_margin.numpy():.1f}")
print(f"Loss' alpha: {model_4.loss.alpha.numpy():.1f}")

Model accuracy: 0.9602
Model's mean certificate: 0.7708
Loss' minimum margin: 0.2
Loss' alpha: 50.0


We confirmed experimentally the accuracy-robustness trade-off: a higher minimal margin
and a lower alpha increases robustness, but also decreases accuracy.


## 🎉 Congratulations

You now know how to train provable adversarially robust 1-Lipschitz neural networks!

👓 Interested readers can learn more about the role of loss functions and the
accuracy-robustness trade-off which occurs when training adversarially robust
1-Lipschitz neural network in the following paper:  
 [Pay attention to your loss: understanding misconceptions about 1-Lipschitz neural networks](https://arxiv.org/abs/2104.05097).
